In [2]:
import os
import glob
import numpy as np
import librosa
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.cluster import KMeans
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
from scipy.signal import welch
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from sklearn.manifold import TSNE
from umap import UMAP

# Directory paths
audio_dir = "/content/drive/MyDrive/Colab Notebooks/piping_database/FinalDataSet/"
output_csv = "/content/drive/MyDrive/Colab Notebooks/piping_database/bee_sound_Cluster_labels.csv"

# Label Mapping
label_mapping = {
    0: ("Normal", "Not Stressed"),
    1: ("MissingQueenBee", "Stressed"),
    2: ("Swarming", "Stressed"),
    3: ("Hyper", "Stressed")
}

# Function to extract enhanced audio features
def extract_audio_features(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None)

        # MFCCs
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)

        # Spectral Features
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr).mean()
        spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr).mean()
        rms = librosa.feature.rms(y=y).mean()

        # PSD Features
        freqs, psd = welch(y, fs=sr, nperseg=1024)
        band_definitions = [(300, 500), (300, 350), (300, 3600), (100, 2000), (200, 350)]
        psd_band_energy = [np.sum(psd[(freqs >= band[0]) & (freqs <= band[1])]) for band in band_definitions]

        return np.hstack([
            mfcc_mean,
            *psd_band_energy,
            spectral_bandwidth,
            spectral_contrast,
            rms
        ])
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Process audio files
def process_audio_files(audio_dir):
    features = []
    file_names = []
    for file_path in glob.glob(f"{audio_dir}/**/*.*", recursive=True):
        if file_path.lower().endswith(('.wav', '.mp3')):
            print(f"Processing {file_path}...")
            feature_vector = extract_audio_features(file_path)
            if feature_vector is not None:
                features.append(feature_vector)
                file_names.append(os.path.basename(file_path))
    return np.array(features), file_names

# Build an autoencoder
def build_autoencoder(input_dim, encoding_dim, loss_function):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)

    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)
    autoencoder.compile(optimizer='adam', loss=loss_function)
    return autoencoder, encoder

# Compare clustering results for different loss functions
def compare_loss_functions(features, n_clusters=4, encoding_dim=20):
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)

    input_dim = features_scaled.shape[1]
    loss_functions = {
        'MSE': MeanSquaredError(),
        'MAE': MeanAbsoluteError()
    }

    results = {}

    for name, loss_function in loss_functions.items():
        print(f"Training autoencoder with {name} loss...")
        autoencoder, encoder = build_autoencoder(input_dim, encoding_dim, loss_function)
        autoencoder.fit(features_scaled, features_scaled, epochs=50, batch_size=32, shuffle=True, verbose=0)

        latent_features = encoder.predict(features_scaled)
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        cluster_labels = kmeans.fit_predict(latent_features)
        silhouette_avg = silhouette_score(latent_features, cluster_labels)
        davies_bouldin_avg = davies_bouldin_score(latent_features, cluster_labels)

        results[name] = {
            'latent_features': latent_features,
            'cluster_labels': cluster_labels,
            'silhouette_score': silhouette_avg,
            'davies_bouldin_index': davies_bouldin_avg
        }

    return results

# Save labels and features for CNN training
def save_labels_and_features(features, labels, output_csv):
    labeled_data = pd.DataFrame(features)
    labeled_data['label'] = labels
    labeled_data.to_csv(output_csv, index=False)
    print(f"Labels and features saved to {output_csv}")

# Build and train CNN model
def build_cnn(input_shape, num_classes):
    model = Sequential([
        Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        Conv1D(128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Main workflow
def main():
    print("Processing audio files...")
    features, file_names = process_audio_files(audio_dir)

    if features.size > 0:
        # Normalize features
        scaler = MinMaxScaler()
        features_scaled = scaler.fit_transform(features)

        # Clustering for pseudo-labeling
        kmeans = KMeans(n_clusters=4, random_state=42)
        cluster_labels = kmeans.fit_predict(features_scaled)

        # Save labeled data
        save_labels_and_features(features_scaled, cluster_labels, output_csv)

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(features_scaled, cluster_labels, test_size=0.2, random_state=42)

        # Reshape for CNN input
        X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
        X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

        # One-hot encode labels
        y_train_encoded = to_categorical(y_train, num_classes=4)
        y_test_encoded = to_categorical(y_test, num_classes=4)

        # Build and train CNN
        input_shape = (X_train_reshaped.shape[1], X_train_reshaped.shape[2])
        cnn_model = build_cnn(input_shape, num_classes=4)
        history = cnn_model.fit(
            X_train_reshaped, y_train_encoded,
            validation_data=(X_test_reshaped, y_test_encoded),
            epochs=30, batch_size=32, verbose=1
        )

        # Evaluate the CNN
        test_loss, test_accuracy = cnn_model.evaluate(X_test_reshaped, y_test_encoded, verbose=0)
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    else:
        print("No valid audio files processed.")

if __name__ == "__main__":
    main()


Processing audio files...
Processing /content/drive/MyDrive/Colab Notebooks/piping_database/FinalDataSet/2022-06-05--17-41-01_2__segment0.wav...
Processing /content/drive/MyDrive/Colab Notebooks/piping_database/FinalDataSet/2022-06-05--17-41-01_2__segment5.wav...
Processing /content/drive/MyDrive/Colab Notebooks/piping_database/FinalDataSet/2022-06-05--17-41-01_2__segment2.wav...
Processing /content/drive/MyDrive/Colab Notebooks/piping_database/FinalDataSet/2022-06-05--18-40-08_2__segment2.wav...
Processing /content/drive/MyDrive/Colab Notebooks/piping_database/FinalDataSet/2022-06-05--17-41-01_2__segment4.wav...
Processing /content/drive/MyDrive/Colab Notebooks/piping_database/FinalDataSet/2022-06-05--19-39-14_2__segment0.wav...
Processing /content/drive/MyDrive/Colab Notebooks/piping_database/FinalDataSet/2022-06-05--18-40-08_2__segment0.wav...
Processing /content/drive/MyDrive/Colab Notebooks/piping_database/FinalDataSet/2022-06-05--18-40-08_2__segment5.wav...
Processing /content/dr

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.4426 - loss: 1.2990 - val_accuracy: 0.4641 - val_loss: 1.2041
Epoch 2/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5308 - loss: 1.1336 - val_accuracy: 0.4641 - val_loss: 1.0838
Epoch 3/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5345 - loss: 1.0179 - val_accuracy: 0.7582 - val_loss: 0.8276
Epoch 4/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7733 - loss: 0.7480 - val_accuracy: 0.8431 - val_loss: 0.5102
Epoch 5/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8629 - loss: 0.5333 - val_accuracy: 0.8562 - val_loss: 0.3583
Epoch 6/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8864 - loss: 0.3447 - val_accuracy: 0.9346 - val_loss: 0.2519
Epoch 7/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8931 - loss: 0.2891 - val_accuracy: 0.9542 - val_loss: 0.1822
Epoch 8/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9364 - loss: 0.2360 - val_accuracy: 0.9412 - val_loss

In [1]:
!pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.7 MB/s eta 0:00:00
